##掛載 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##加入專案路徑

In [ ]:
import os
import sys
from flask import Flask, request, abort

# 🌟 0. 環境設定與診斷 (強制在匯入前執行) 🌟
# 請確保 Cell 1: drive.mount('/content/drive') 已經成功運行

PROJECT_PATH = '/content/drive/MyDrive/projects/LINE BOT'

# 診斷：檢查檔案是否真的存在於 Python 視角下的路徑
FILE_CHECK_NAME = 'v1_sqlite_dataseat.py'
FILE_CHECK_PATH = os.path.join(PROJECT_PATH, FILE_CHECK_NAME)

# --- 診斷邏輯 ---
if os.path.exists(FILE_CHECK_PATH):
    print(f"✅ 診斷成功：文件 '{FILE_CHECK_NAME}' 存在於 {PROJECT_PATH}。")
    if PROJECT_PATH not in sys.path:
        sys.path.append(PROJECT_PATH)
        print(f"✅ 已將路徑加入系統搜尋清單。")
else:
    # 如果文件不存在，立即停止並報告錯誤
    print(f"❌ 診斷失敗：文件 '{FILE_CHECK_NAME}' 不存在於 {PROJECT_PATH}。")
    print("請仔細檢查 Google Drive 上的資料夾名稱是否有隱藏的空格或符號。")
    raise FileNotFoundError("路徑或檔案名稱錯誤，無法繼續。")
# --- 診斷結束 ---

# 1. 導入您的資料庫函式 (現在路徑已設定，匯入應該成功)
from v1_sqlite_dataseat import init_db, insert_log, close_db, get_user_log_count

# ... (LINE Bot SDK 導入和 Flask 程式碼接在後面)
# ...

# (請確保 Flask/LINE Bot 程式碼是接在後面，而不是只有這段)

##1) 安裝套件（指定版本範圍＋用途註解）

In [ ]:
# 安裝 LINE Bot SDK v3（官方最新主線）
!pip install "line-bot-sdk>=3.0.0,<4.0.0"

# 安裝 Flask（Web 框架，提供 webhook 端點）
!pip install "Flask>=3.0.0,<4.0.0"

# 安裝 pyngrok（把本機/Colab 5000 port 映射成公開 HTTPS）
!pip install "pyngrok>=7.0.0,<8.0.0"

##2) 開 ngrok（先執行這一格拿到公開網址）

In [ ]:
from pyngrok import ngrok

# 換成你自己的 ngrok authtoken
NGROK_AUTHTOKEN = "你的密碼"   # 例：2pEWt4...（不要公開在 GitHub）
PORT = 5000

# 設定金鑰
ngrok.set_auth_token(NGROK_AUTHTOKEN)

# 關閉舊隧道（避免重複）
for t in ngrok.get_tunnels():
    ngrok.disconnect(t.public_url)

# 建立新隧道，取得公開 HTTPS 網址
public_url = ngrok.connect(PORT, "http").public_url

print("👉 外網網址：", public_url)
print("Webhook URL：", public_url + "/callback")

##3) LINE Bot 程式

In [ ]:
#從Line bot sdk pypl網站找到程式碼貼上
#https://pypi.org/project/line-bot-sdk/
import os
import sys
import re # 🌟 關鍵：導入正規表達式模組 🌟
from flask import Flask, request, abort

from v1_sqlite_dataseat import (
    init_db,
    insert_log,
    close_db,
    get_user_log_count,
    # 🌟 修正：導入所有需要的 CRUD 函式 🌟
    get_latest_log,
    get_first_log,
    get_nth_log,
    delete_all_logs,
    delete_log_by_id,
    get_all_logs_summary
)

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent
)


app = Flask(__name__)

# 憑證設定：請替換成您的 LINE Bot 憑證
CHANNEL_ACCESS_TOKEN = os.environ.get('LINE_ACCESS_TOKEN', 'YOUR_LINE_ACCESS_TOKEN')
CHANNEL_SECRET = os.environ.get('LINE_CHANNEL_SECRET', 'YOUR_LINE_CHANNEL_SECRET')

configuration = Configuration(access_token=CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(CHANNEL_SECRET)

# 🌟 數據庫優化：註冊資料庫關閉函式 🌟
# 確保每個請求結束後都會關閉連線，釋放資源
app.teardown_appcontext(close_db)

QUERY_PATTERN = re.compile(r'^(?:第\s*|查詢\s*第?\s*)(\d+)\s*篇?') # 查詢第 N 篇
DELETE_ALL_PATTERN = re.compile(r'^(?:刪除所有|清除所有|/clear|/delete\s*all)$', re.I) # 刪除所有
DELETE_ID_PATTERN = re.compile(r'^(?:刪除\s*ID|delete\s*id)\s*[:：]?\s*(\d+)$', re.I) # 刪除 ID

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    user_id = event.source.user_id
    user_input = event.message.text.strip()
    reply_text = None

    # 🌟 新增：二次確認的關鍵訊息 🌟
    CONFIRM_DELETE_MSG = "請再次輸入『確定刪除』來清空所有日記。"
    # ----------------------------------------------------
    # A. 處理「刪除」指令
    # ----------------------------------------------------
    # 1. 處理「確認刪除」回覆（只有在收到這個關鍵字時才執行清空）
    if user_input == "確定刪除":
        # 檢查使用者是否剛剛被要求確認（雖然簡單，但在單一流程中有效）
        # 這裡我們信任使用者只有在被要求時才會輸入 '確定刪除'
        deleted_count = delete_all_logs(user_id)
        if deleted_count > 0:
            reply_text = (
                f"🗑️ 警告：已成功刪除您共 {deleted_count} 篇日記！您的紀錄已清空。\n"
                f"現在您可以重新開始寫日記了！"
            )
        else:
            # 可能是誤觸，或者本來就沒有紀錄
            reply_text = "您的日記庫中沒有任何紀錄可以刪除，不需要進行清空操作。"

    # 2. 處理「刪除 ID」指令
    elif (match := DELETE_ID_PATTERN.match(user_input.lower())):
        log_id_to_delete = int(match.group(1))

        if delete_log_by_id(log_id_to_delete):
            total_logs = get_user_log_count(user_id)
            reply_text = (
                f"🗑️ 已成功刪除唯一 ID 為 {log_id_to_delete} 的紀錄！\n"
                f"目前您還剩下 {total_logs} 篇日記。"
            )
        else:
            reply_text = f"找不到 ID 為 {log_id_to_delete} 的紀錄可供刪除，或該紀錄不屬於您。"

    # 3. 處理「請求刪除所有」指令 (觸發二次確認)
    elif DELETE_ALL_PATTERN.match(user_input.lower()):
        total_logs = get_user_log_count(user_id)
        if total_logs == 0:
            reply_text = "您的日記庫中沒有任何紀錄可以刪除。"
        else:
            # 🌟 關鍵：要求二次確認 🌟
            reply_text = (
                f"⚠️ **您確定要刪除所有 {total_logs} 篇日記嗎？**\n"
                f"此操作不可復原！\n\n"
                f"{CONFIRM_DELETE_MSG}"
            )


    # ----------------------------------------------------
    # B. 處理「清單/預覽所有」指令 (List/Preview All)
    # ----------------------------------------------------
    elif user_input.lower() in ['/list', '清單', '預覽所有']:
        summaries = get_all_logs_summary(user_id)
        total_logs = len(summaries)

        if total_logs > 0:
            list_items = []
            # i 是從 0 開始的索引
            for i, (log_id, date_part, summary) in enumerate(summaries):
                # 顯示格式: [編號] (日期) ID: XXX | 摘要內容
                list_items.append(
                    f"[{i+1}] ({date_part}) ID:{log_id} | {summary}"
                )

            reply_text = (
                f"📝 您的日記清單 (共 {total_logs} 篇)：\n"
                f"--------------------------------------\n"
                + "\n".join(list_items) +
                f"\n--------------------------------------\n"
                f"可輸入『刪除 ID: XXX』或『第 N 篇』進行操作。\n"
                f"如需全部刪除，請輸入『刪除所有』或『/clear』"
            )
        else:
            reply_text = "您的日記庫中沒有任何紀錄喔！"

    # ----------------------------------------------------
    # C. 處理「查詢單篇」指令 (Latest/First/Nth)
    # ----------------------------------------------------

    # 只有在 reply_text 仍為 None 時，才進入查詢邏輯
    elif reply_text is None:
        log = None
        title = None
        n = None

        if user_input.lower() in ['/last', '最後一篇', '查詢最新']:
            log = get_latest_log(user_id)
            title = "最新"

        elif user_input.lower() in ['/first', '第一篇', '查詢最早']:
            log = get_first_log(user_id)
            title = "第一篇"

        elif (match := QUERY_PATTERN.match(user_input)):
            n = int(match.group(1))
            log = get_nth_log(user_id, n)
            title = f"第 {n}"

        # 組合查詢單篇結果的回覆 (預覽功能)
        if log:
            log_id, log_text = log # 🌟 解構元組，獲得 ID 和內容 🌟
            reply_text = (
                f"📜 您寫下的**{title}**紀錄是：\n"
                f"👉 唯一 ID: {log_id} 👈\n"
                f"--------------------\n"
                f"「{log_text}」\n"
                f"--------------------\n"
                f"如需刪除此篇，請輸入： 刪除 ID: {log_id}"
            )
        elif title: # 命中了查詢指令，但沒找到紀錄
            total_logs = get_user_log_count(user_id)
            if total_logs > 0 and n:
                reply_text = f"目前您只寫了 {total_logs} 篇紀錄，無法查詢第 {n} 篇。"
            elif total_logs == 0:
                reply_text = "您還沒有寫過任何紀錄喔！"

    # ----------------------------------------------------
    # D. 處理「寫日記」指令 (如果不是任何指令，則儲存)
    # ----------------------------------------------------
    if reply_text is None:
        # 儲存日記
        insert_log(user_id=user_id, raw_text=user_input)
        total_logs = get_user_log_count(user_id)

        reply_text = (
            f"📝 您的紀錄已成功收錄！\n"
            f"這是您寫下的第 {total_logs} 篇紀錄。\n"
            f"輸入『/list』即可預覽所有紀錄清單！"
        )

    # ----------------------------------------------------
    # E. 回覆使用者
    # ----------------------------------------------------
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=reply_text)]
            )
        )

if __name__ == "__main__":
    # 應用程式啟動前，先初始化資料庫
    init_db()

    # 運行 Flask 服務
    app.run(port=5000)
